# TFL - Accessibility

In [1]:
#! pip install requests pandas matplotlib python-dotenv

In [36]:
import requests
import pandas as pd
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
from dotenv import load_dotenv

* _load variables_

In [3]:
load_dotenv()

True

In [4]:
APP_ID = os.getenv('APP_ID')
API_KEY = os.getenv('API_KEY')
YEAR = 2019

* _step free interchanges_

In [5]:
df = pd.read_csv('data/StepFreeIntechangeInfo.csv')

df.head()

,FromPlatformUniqueId,ToPlatformUniqueId,DistanceInMetres
0,910GACTNCTL-Plat01-WB-london-overground,910GACTNCTL-Plat02-EB-london-overground,50
1,910GACTNCTL-Plat02-EB-london-overground,910GACTNCTL-Plat01-WB-london-overground,50
2,910GANERLEY-Plat01-NB-london-overground,910GANERLEY-Plat02-SB-london-overground,220
3,910GANERLEY-Plat02-SB-london-overground,910GANERLEY-Plat01-NB-london-overground,220
4,910GBHILLPK-Plat01-SB-london-overground,910GBHILLPK-Plat02-NB-london-overground,290


* _station lifts_

In [6]:
df_2 = pd.read_csv('data/Lifts.csv')

In [7]:
lifts_cleaned = df_2.drop(columns=['NationalRailUprn', 'LiftName', 'IntermediateAreas', 'IntermediateAreas2'], axis=1)

lifts_cleaned.head()

,StationUniqueId,LiftUniqueId,LiftId,FriendlyName,FromAreas,ToAreas,LimitedCapacityLift,LiftNotes
0,910GACTONML,910GACTONML-Lift-1,1,Lift A,910GACTONML-1001003-AC-S-6,910GACTONML-1001003-RPL1-2,False,NaN
1,910GACTONML,910GACTONML-Lift-2,2,Lift B,910GACTONML-1001003-AC-S-6,910GACTONML-1001003-RPL2-5,False,NaN
2,910GBRENTX,910GBRENTX-Lift-1,1,Lift 1,910GBRENTX-1002310-EntW-7,910GBRENTX-1002310-FB-8,False,NaN
3,910GBRENTX,910GBRENTX-Lift-2,2,Lift 2,910GBRENTX-1002310-RLY-5,910GBRENTX-1002310-RPL4-10|910GBRENTX-1002310-...,False,NaN
4,910GBRENTX,910GBRENTX-Lift-3,3,Lift 3,910GBRENTX-1002310-RLY-5,910GBRENTX-1002310-RPL2-3|910GBRENTX-1002310-R...,False,NaN


* _stations for StationUniqueId_

In [8]:
df_3 = pd.read_csv('data/Stations.csv')


In [9]:
stations_info = df_3.drop(columns=['HubNaptanCode', 'OutsideStationUniqueId', 'BlueBadgeCarParkSpaces', 'TaxiRanksOutsideStation', 'MainBusInterchange', 'PierInterchange', 'NationalRailInterchange', 'AirportInterchange', 'EmiratesAirLineInterchange'], axis=1)

stations_info.rename(columns={"UniqueId": "StationUniqueId"}, inplace=True)

stations_info.head()

,StationUniqueId,Name,FareZones,Wifi,BlueBadgeCarParking
0,HUBABW,Abbey Wood,4,False,False
1,910GACTNCTL,Acton Central,3,True,False
2,910GACTONML,Acton Main Line,3,False,False
3,910GANERLEY,Anerley,4,True,False
4,910GBCKNHMH,Beckenham Hill,4,False,False


* _staionsinfo merge with liftscleaned_

In [10]:
stations_with_lifts = stations_info.merge(lifts_cleaned, on='StationUniqueId', how='left').drop_duplicates()

stations_with_lifts.head()

,StationUniqueId,Name,FareZones,Wifi,BlueBadgeCarParking,LiftUniqueId,LiftId,FriendlyName,FromAreas,ToAreas,LimitedCapacityLift,LiftNotes
0,HUBABW,Abbey Wood,4,False,False,HUBABW-Lift-1,1,Lift A,HUBABW-1001001-FELIX-6,HUBABW-1001001-HARRO-2,False,NaN
1,HUBABW,Abbey Wood,4,False,False,HUBABW-Lift-2,2,Lift B,HUBABW-1001001-FELIX-6,NaN,False,NaN
2,HUBABW,Abbey Wood,4,False,False,HUBABW-Lift-3,3,Lift C,HUBABW-1001001-WILTO-1,HUBABW-1001001-HARRO-2,False,NaN
3,HUBABW,Abbey Wood,4,False,False,HUBABW-Lift-4,4,Lift D,HUBABW-1001001-WILTO-1,NaN,False,NaN
4,HUBABW,Abbey Wood,4,False,False,HUBABW-Lift-6,6,Lift F,HUBABW-1001001-TH-11,HUBABW-1001001-RPLD-3|HUBABW-1001001-RPLC-7,False,NaN


* _platform services_
    * _looking for step free links with uniqueid for station_

In [11]:
df_4 = pd.read_csv('data/PlatformServices.csv')


In [12]:
services_cleaned = df_4.drop(columns=['MinGap', 'MaxGap', 'AverageGap', 'MinStep', 'AverageStep', 'DesignatedLevelAccessPoint', 'LocationOfLevelAccess', 'GroupName'], axis=1)

services_cleaned.rename(columns={'PlatformUniqueId' : 'FromPlatformUniqueId'}, inplace=True)

services_cleaned.head()

,FromPlatformUniqueId,StopAreaNaptanCode,Line,DirectionTowards,MaxStep,LevelAccessByManualRamp,AdditionalAccessibilityInformation
0,HUBABW-Plat01-WB-national-rail,910GABWD,national-rail,NaN,NaN,True,NaN
1,HUBABW-Plat02-EB-national-rail,910GABWD,national-rail,NaN,NaN,True,NaN
2,HUBABW-Plat03-WB-elizabeth,910GABWD,elizabeth,NaN,NaN,True,NaN
3,HUBABW-Plat04-WB-elizabeth,910GABWD,elizabeth,NaN,NaN,True,NaN
4,910GACTNCTL-Plat01-WB-london-overground,910GACTNCTL,mildmay,Richmond,NaN,True,Step-free platform interchange requires a 50m ...


In [13]:
step_free_platforms = df.merge(services_cleaned, on='FromPlatformUniqueId', how='left').drop_duplicates()

step_free_platforms.head()

,FromPlatformUniqueId,ToPlatformUniqueId,DistanceInMetres,StopAreaNaptanCode,Line,DirectionTowards,MaxStep,LevelAccessByManualRamp,AdditionalAccessibilityInformation
0,910GACTNCTL-Plat01-WB-london-overground,910GACTNCTL-Plat02-EB-london-overground,50,910GACTNCTL,mildmay,Richmond,NaN,True,Step-free platform interchange requires a 50m ...
1,910GACTNCTL-Plat02-EB-london-overground,910GACTNCTL-Plat01-WB-london-overground,50,910GACTNCTL,mildmay,Stratford,NaN,True,NaN
2,910GANERLEY-Plat01-NB-london-overground,910GANERLEY-Plat02-SB-london-overground,220,910GANERLEY,windrush,Dalston Junction,NaN,True,NaN
3,910GANERLEY-Plat02-SB-london-overground,910GANERLEY-Plat01-NB-london-overground,220,910GANERLEY,windrush,West Croydon,NaN,True,NaN
4,910GBHILLPK-Plat01-SB-london-overground,910GBHILLPK-Plat02-NB-london-overground,290,910GBHILLPK,weaver,NaN,NaN,True,Use the St Mark's Road entrance for southbound...


* _strip stepfree platforms['FromPlatformUniqueId'] to correlate with stationsinfo['stationuniqueid']_

In [14]:
# use regex to strip numbers and extract first part of str pre '-'
step_free_platforms['StationUniqueId'] = step_free_platforms['FromPlatformUniqueId'].str.split("-").str[0]

step_free_platforms.head()


,FromPlatformUniqueId,ToPlatformUniqueId,DistanceInMetres,StopAreaNaptanCode,Line,DirectionTowards,MaxStep,LevelAccessByManualRamp,AdditionalAccessibilityInformation,StationUniqueId
0,910GACTNCTL-Plat01-WB-london-overground,910GACTNCTL-Plat02-EB-london-overground,50,910GACTNCTL,mildmay,Richmond,NaN,True,Step-free platform interchange requires a 50m ...,910GACTNCTL
1,910GACTNCTL-Plat02-EB-london-overground,910GACTNCTL-Plat01-WB-london-overground,50,910GACTNCTL,mildmay,Stratford,NaN,True,NaN,910GACTNCTL
2,910GANERLEY-Plat01-NB-london-overground,910GANERLEY-Plat02-SB-london-overground,220,910GANERLEY,windrush,Dalston Junction,NaN,True,NaN,910GANERLEY
3,910GANERLEY-Plat02-SB-london-overground,910GANERLEY-Plat01-NB-london-overground,220,910GANERLEY,windrush,West Croydon,NaN,True,NaN,910GANERLEY
4,910GBHILLPK-Plat01-SB-london-overground,910GBHILLPK-Plat02-NB-london-overground,290,910GBHILLPK,weaver,NaN,NaN,True,Use the St Mark's Road entrance for southbound...,910GBHILLPK


_merge stationsifno and stepfreeplatforms for station name_

In [15]:
info_step_merged = stations_info.merge(step_free_platforms, on='StationUniqueId', how='inner')

info_step_merged.drop(columns='StopAreaNaptanCode', axis=1)

,StationUniqueId,Name,FareZones,Wifi,BlueBadgeCarParking,FromPlatformUniqueId,ToPlatformUniqueId,DistanceInMetres,Line,DirectionTowards,MaxStep,LevelAccessByManualRamp,AdditionalAccessibilityInformation
0,910GACTNCTL,Acton Central,3,True,False,910GACTNCTL-Plat01-WB-london-overground,910GACTNCTL-Plat02-EB-london-overground,50,mildmay,Richmond,NaN,True,Step-free platform interchange requires a 50m ...
1,910GACTNCTL,Acton Central,3,True,False,910GACTNCTL-Plat02-EB-london-overground,910GACTNCTL-Plat01-WB-london-overground,50,mildmay,Stratford,NaN,True,NaN
2,910GANERLEY,Anerley,4,True,False,910GANERLEY-Plat01-NB-london-overground,910GANERLEY-Plat02-SB-london-overground,220,windrush,Dalston Junction,NaN,True,NaN
3,910GANERLEY,Anerley,4,True,False,910GANERLEY-Plat02-SB-london-overground,910GANERLEY-Plat01-NB-london-overground,220,windrush,West Croydon,NaN,True,NaN
4,910GBHILLPK,Bush Hill Park,5,True,True,910GBHILLPK-Plat01-SB-london-overground,910GBHILLPK-Plat02-NB-london-overground,290,weaver,NaN,NaN,True,Use the St Mark's Road entrance for southbound...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,HUBLBG,London Bridge,1,True,False,HUBLBG-Plat04-EB-jubilee,HUBLBG-Plat02-SB-northern,410,jubilee,Stratford,50.0,False,NaN
115,HUBSYD,Sydenham,3,False,False,HUBSYD-Plat01-NB-london-overground,HUBSYD-Plat02-SB-london-overground,350,windrush,Highbury & Islington,NaN,True,Use the Peak Hill Gardens entrance for northbo...
116,HUBSYD,Sydenham,3,False,False,HUBSYD-Plat02-SB-london-overground,HUBSYD-Plat01-NB-london-overground,350,windrush,Crystal Palace and West Croydon,NaN,True,Use the Station Approach emtrance for southbou...
117,HUBWHC,Walthamstow Central,3,True,True,HUBWHC-Plat01-SB-london-overground,HUBWHC-Plat02-NB-london-overground,200,weaver,Liverpool Street,NaN,True,Use the Station Approach entrance for southbou...


In [16]:
info_step_merged.rename(columns={'FromPlatformUniqueId' : 'LiftFromPlatformUniqueId', 'ToPlatformUniqueId' : 'LiftToPlatformUniqueId', 'DistanceInMetres' : 'Distance (m)', 'LevelAccessByManualRamp' : 'Manual Ramp Access'})

,StationUniqueId,Name,FareZones,Wifi,BlueBadgeCarParking,LiftFromPlatformUniqueId,LiftToPlatformUniqueId,Distance (m),StopAreaNaptanCode,Line,DirectionTowards,MaxStep,Manual Ramp Access,AdditionalAccessibilityInformation
0,910GACTNCTL,Acton Central,3,True,False,910GACTNCTL-Plat01-WB-london-overground,910GACTNCTL-Plat02-EB-london-overground,50,910GACTNCTL,mildmay,Richmond,NaN,True,Step-free platform interchange requires a 50m ...
1,910GACTNCTL,Acton Central,3,True,False,910GACTNCTL-Plat02-EB-london-overground,910GACTNCTL-Plat01-WB-london-overground,50,910GACTNCTL,mildmay,Stratford,NaN,True,NaN
2,910GANERLEY,Anerley,4,True,False,910GANERLEY-Plat01-NB-london-overground,910GANERLEY-Plat02-SB-london-overground,220,910GANERLEY,windrush,Dalston Junction,NaN,True,NaN
3,910GANERLEY,Anerley,4,True,False,910GANERLEY-Plat02-SB-london-overground,910GANERLEY-Plat01-NB-london-overground,220,910GANERLEY,windrush,West Croydon,NaN,True,NaN
4,910GBHILLPK,Bush Hill Park,5,True,True,910GBHILLPK-Plat01-SB-london-overground,910GBHILLPK-Plat02-NB-london-overground,290,910GBHILLPK,weaver,NaN,NaN,True,Use the St Mark's Road entrance for southbound...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,HUBLBG,London Bridge,1,True,False,HUBLBG-Plat04-EB-jubilee,HUBLBG-Plat02-SB-northern,410,940GZZLULNB,jubilee,Stratford,50.0,False,NaN
115,HUBSYD,Sydenham,3,False,False,HUBSYD-Plat01-NB-london-overground,HUBSYD-Plat02-SB-london-overground,350,910GSYDENHM,windrush,Highbury & Islington,NaN,True,Use the Peak Hill Gardens entrance for northbo...
116,HUBSYD,Sydenham,3,False,False,HUBSYD-Plat02-SB-london-overground,HUBSYD-Plat01-NB-london-overground,350,910GSYDENHM,windrush,Crystal Palace and West Croydon,NaN,True,Use the Station Approach emtrance for southbou...
117,HUBWHC,Walthamstow Central,3,True,True,HUBWHC-Plat01-SB-london-overground,HUBWHC-Plat02-NB-london-overground,200,910GWLTWCEN,weaver,Liverpool Street,NaN,True,Use the Station Approach entrance for southbou...


In [26]:
info_step_merged['AdditionalAccessibilityInformation'].dropna()

info_step_merged

,StationUniqueId,Name,FareZones,Wifi,BlueBadgeCarParking,FromPlatformUniqueId,ToPlatformUniqueId,DistanceInMetres,StopAreaNaptanCode,Line,DirectionTowards,MaxStep,LevelAccessByManualRamp,AdditionalAccessibilityInformation
0,910GACTNCTL,Acton Central,3,True,False,910GACTNCTL-Plat01-WB-london-overground,910GACTNCTL-Plat02-EB-london-overground,50,910GACTNCTL,mildmay,Richmond,NaN,True,Step-free platform interchange requires a 50m ...
1,910GACTNCTL,Acton Central,3,True,False,910GACTNCTL-Plat02-EB-london-overground,910GACTNCTL-Plat01-WB-london-overground,50,910GACTNCTL,mildmay,Stratford,NaN,True,NaN
2,910GANERLEY,Anerley,4,True,False,910GANERLEY-Plat01-NB-london-overground,910GANERLEY-Plat02-SB-london-overground,220,910GANERLEY,windrush,Dalston Junction,NaN,True,NaN
3,910GANERLEY,Anerley,4,True,False,910GANERLEY-Plat02-SB-london-overground,910GANERLEY-Plat01-NB-london-overground,220,910GANERLEY,windrush,West Croydon,NaN,True,NaN
4,910GBHILLPK,Bush Hill Park,5,True,True,910GBHILLPK-Plat01-SB-london-overground,910GBHILLPK-Plat02-NB-london-overground,290,910GBHILLPK,weaver,NaN,NaN,True,Use the St Mark's Road entrance for southbound...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,HUBLBG,London Bridge,1,True,False,HUBLBG-Plat04-EB-jubilee,HUBLBG-Plat02-SB-northern,410,940GZZLULNB,jubilee,Stratford,50.0,False,NaN
115,HUBSYD,Sydenham,3,False,False,HUBSYD-Plat01-NB-london-overground,HUBSYD-Plat02-SB-london-overground,350,910GSYDENHM,windrush,Highbury & Islington,NaN,True,Use the Peak Hill Gardens entrance for northbo...
116,HUBSYD,Sydenham,3,False,False,HUBSYD-Plat02-SB-london-overground,HUBSYD-Plat01-NB-london-overground,350,910GSYDENHM,windrush,Crystal Palace and West Croydon,NaN,True,Use the Station Approach emtrance for southbou...
117,HUBWHC,Walthamstow Central,3,True,True,HUBWHC-Plat01-SB-london-overground,HUBWHC-Plat02-NB-london-overground,200,910GWLTWCEN,weaver,Liverpool Street,NaN,True,Use the Station Approach entrance for southbou...


_stations with AdditionalAccessibilityInformation_

In [32]:
accessibility_notes = info_step_merged.dropna(subset=['Name', 'AdditionalAccessibilityInformation'])

accessibility_notes.head()

,StationUniqueId,Name,FareZones,Wifi,BlueBadgeCarParking,FromPlatformUniqueId,ToPlatformUniqueId,DistanceInMetres,StopAreaNaptanCode,Line,DirectionTowards,MaxStep,LevelAccessByManualRamp,AdditionalAccessibilityInformation
0,910GACTNCTL,Acton Central,3,True,False,910GACTNCTL-Plat01-WB-london-overground,910GACTNCTL-Plat02-EB-london-overground,50,910GACTNCTL,mildmay,Richmond,NaN,True,Step-free platform interchange requires a 50m ...
4,910GBHILLPK,Bush Hill Park,5,True,True,910GBHILLPK-Plat01-SB-london-overground,910GBHILLPK-Plat02-NB-london-overground,290,910GBHILLPK,weaver,NaN,NaN,True,Use the St Mark's Road entrance for southbound...
5,910GBHILLPK,Bush Hill Park,5,True,True,910GBHILLPK-Plat02-NB-london-overground,910GBHILLPK-Plat01-SB-london-overground,290,910GBHILLPK,weaver,NaN,NaN,True,Use the Queen Anne's Place entrance for northb...
6,910GBROCKLY,Brockley,2,True,False,910GBROCKLY-Plat01-NB-london-overground,910GBROCKLY-Plat02-SB-london-overground,50,910GBROCKLY,windrush,Dalston Junction and Highbury & Islington,NaN,True,Step-free platform interchange requires a 500m...
7,910GBROCKLY,Brockley,2,True,False,910GBROCKLY-Plat02-SB-london-overground,910GBROCKLY-Plat01-NB-london-overground,50,910GBROCKLY,windrush,Crystal Palace and West Croydon,NaN,True,Step-free platform interchange requires a 500m...


In [33]:
for name, notes in zip(accessibility_notes['Name'], accessibility_notes['AdditionalAccessibilityInformation']):
    print(f'{name}: Accessibility Notes: {notes}')

Acton Central: Accessibility Notes: Step-free platform interchange requires a 50m journey via street. Use East Churchfield Road entrance for westbound and Churchfield Road for eastbound
Bush Hill Park: Accessibility Notes: Use the St Mark's Road entrance for southbound platform. Step-free platform interchange requires a 290m journey via street
Bush Hill Park: Accessibility Notes: Use the Queen Anne's Place entrance for northbound platform. Step-free platform interchange requires a 290m journey via street
Brockley: Accessibility Notes: Step-free platform interchange requires a 500m journey via street
Brockley: Accessibility Notes: Step-free platform interchange requires a 500m journey via street
Hackney Downs: Accessibility Notes: Step-free access to/from platform 1 is via Hackney Central entrance, interchange footpath and lift
Highams Park: Accessibility Notes: Use the Wilton Place entrance for northbound platform. Step-free platform interchange requires a 350m journey via street
Higha

_lines with the most lifts to platform_